In [1]:
from __future__ import unicode_literals
from matplotlib import pyplot as plt
import pickle
import pandas as pd
import numpy as np
import nltk as nk
import untangle
import seaborn as sns
import operator
import string 
nk.download('punkt')
nk.download('stopwords')
nk.download('wordnet')
nk.download('averaged_perceptron_tagger')

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno -3] Temporary failure in name resolution>


False

## Asociación de palabras

# bigramcollocationfinder.from_words

#### 1. Levantar el corpus AP, separando cada noticia como un elemento distinto en un diccionario (DOCNO :TEXT).


In [2]:
corpus_ap = {}
obj = untangle.parse('ap/ap.txt')
for i in range(len(obj.root.DOC)):
    documento = obj.root.DOC[i]
    corpus_ap[documento.DOCNO.cdata.strip()] = documento.TEXT.cdata.strip()

### 2. Calcular el tamaño del vocabulario

#### Pasos de filtrado

1. Tokenizamos oraciones
2. Para cada oración realizamos una tokenización de palabras básica (separamos por espacios)
3. Computamos el POS tag de cada palabra
4. Usando el POS tag lematizamos cada palabra
5. Para cada palabra lematizada aumentamos en 1 la cantidad de veces que aparece

In [3]:
def clean_chars(word):
    replaceable = [",", ".", "-", "_", "`", ":", ";", "!", "?", "`", "'"]
    for token in replaceable:
        word = word.replace(token, " ")
    return word.strip()

stopw = set(nk.corpus.stopwords.words("english"))


documents = []

for (docno, text) in corpus_ap.items():
    tokens = nk.sent_tokenize(text.lower())
    document = ""
    for sent in tokens:
        document += sent
    documents.append(document)

len(documents)
    
words_by_doc = [[clean_chars(w) for w in d.split() if len(clean_chars(w)) > 1 or clean_chars(w).isalpha()] 
                  for d in documents]
        
words_by_doc = [[w for w in sent if w not in stopw and len(w) > 1] for sent in words_by_doc]

In [4]:
word_semantic = nk.tag.pos_tag_sents(words_by_doc)

In [5]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return nk.corpus.wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return nk.corpus.wordnet.VERB
    elif treebank_tag.startswith('N'):
        return nk.corpus.wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return nk.corpus.wordnet.ADV
    else:
        return nk.corpus.wordnet.NOUN

lemmatizer = nk.WordNetLemmatizer()

word_count = {}

clean_texts = []

for document in word_semantic:
    text = ""
    for (word, tag) in document:
        lemmatized = lemmatizer.lemmatize(word, get_wordnet_pos(tag))
        word = lemmatized.split()[0]
        if not word[0].isalpha():
            continue
        elif word not in word_count:
            word_count[word] = 1
        else:
            word_count[word] += 1
        text += " " + word
    clean_texts.append(text)
    
with open('word_count2.pickle', 'wb') as handle:
    pickle.dump(word_count, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
with open('word_count2.pickle', 'rb') as handle:
    word_count = pickle.load(handle) 

word_count = [(x, y) for (x, y) in sorted(word_count.items(), key=operator.itemgetter(1), reverse=True)]

In [7]:
word_count[:500]

[(u'say', 10730),
 (u'said', 3534),
 (u'year', 2622),
 (u'would', 2561),
 (u'state', 2081),
 (u'new', 1993),
 (u'u', 1957),
 (u'percent', 1942),
 (u'one', 1705),
 (u'people', 1663),
 (u'million', 1630),
 (u'two', 1608),
 (u'government', 1556),
 (u'also', 1535),
 (u'president', 1513),
 (u'make', 1478),
 (u'last', 1373),
 (u'official', 1353),
 (u'take', 1333),
 (u'go', 1325),
 (u'report', 1266),
 (u'time', 1236),
 (u'first', 1144),
 (u'soviet', 1109),
 (u'include', 1056),
 (u'could', 1050),
 (u'day', 1045),
 (u'get', 1005),
 (u'company', 963),
 (u'week', 960),
 (u'three', 956),
 (u'tell', 949),
 (u'american', 941),
 (u'bush', 934),
 (u'use', 926),
 (u'work', 905),
 (u'billion', 894),
 (u'police', 894),
 (u'call', 887),
 (u'today', 878),
 (u'give', 857),
 (u'come', 855),
 (u'country', 845),
 (u'group', 836),
 (u'force', 833),
 (u'plan', 826),
 (u'month', 818),
 (u'national', 818),
 (u'high', 813),
 (u'city', 807),
 (u'court', 803),
 (u'house', 802),
 (u'united', 792),
 (u'thursday', 792),

In [8]:
print "Tamaño del vocabulario: ", len(word_count), " palabras."

Tamaño del vocabulario:  29435  palabras.


#### 3. Para las 500 palabras con más apariciones, calcular el par más asociado según la medida presentada.

In [9]:
most_used = word_count[:500]

Para el cálculo utilizaremos una ventana de 5 palabras como indica el paper.

In [15]:
def get_close_words_flex(words, idx):
    close_izq = words[max(0, idx-10):idx]
    close_der = words[idx+1:min(len(words)-1,idx + 11)]
    i=0
    while i<len(close_izq):
        if words[idx]==close_izq[i]:
            close_izq=close_izq[i+6:]
            i=0
        i+=1
    i=0
    while i<len(close_der):
        if words[idx]==close_der[i]:
            close_der=close_der[:i]
            i=0
        i+=1
    while (len(close_izq)>5):
        close_izq = close_izq[1:]
    if len(close_der)>5:
        close_der = close_der[:5]
    return close_izq + close_der

def get_close_words(words, idx):
    return words[max(0, idx-5):idx] + words[idx+1:min(len(words)-1,idx + 6)]

window_size = 5

asociaciones = {}
for word,amt in most_used:
        asociaciones[word] = {}
for document in clean_texts:
    for (target, amt) in most_used:
        sdoc = document.split()
        for idx in range(len(sdoc)):
            if sdoc[idx] == target:
#                for pareja in get_close_words(sdoc, idx):
                for pareja in get_close_words_flex(sdoc, idx):
                    asociaciones[target][pareja] = \
                        asociaciones[target][pareja] + 1 if pareja in asociaciones[target] else 1

In [16]:
palabras_totales = sum([y for (x, y)  in word_count])
world_count_dict = dict(word_count)

def freq(x):
    return world_count_dict[x] / float(palabras_totales)

def info_mutua(x, y):       
    if y not in asociaciones[x]:
        return 0
    else:
        # print float(asociaciones[x][y])/palabras_totales, freq(x), freq(y)
        return (float(asociaciones[x][y])/palabras_totales) / (freq(x)*freq(y))

Al igual que en el paper citado, no tomamos en cuenta las duplas de palabras con muy pocas apariciones, ya que vuelven inestable la medición.

In [17]:
maximas_relacionadas = {}
for palabra_estrella, cant in most_used:
    maximas_relacionadas[palabra_estrella] = ('', 0)
    palabras_relacionadas = asociaciones[palabra_estrella].keys()
    #print len(palabras_relacionadas)
    for palabra_relacionada in palabras_relacionadas:
        i_mutua = info_mutua(palabra_estrella, palabra_relacionada)
        # ahora revisamos que aparezca una cantidad mínima de 5 veces esta dupla
        valida = asociaciones[palabra_estrella][palabra_relacionada]>5
        if maximas_relacionadas[palabra_estrella][1] < i_mutua and valida:
            maximas_relacionadas[palabra_estrella] = (palabra_relacionada, i_mutua)
maximas_relacionadas

{u'accord': (u'sources', 524.0467098533925),
 u'accuse': (u'conspire', 871.3316326530612),
 u'across': (u'shower', 487.9457142857143),
 u'act': (u'specie', 408.24143426294825),
 u'action': (u'disciplinary', 977.1322314049588),
 u'add': (u'anything', 87.00000000000001),
 u'administration': (u'aeronautics', 922.5864345738296),
 u'africa': (u'namibia', 1196.4745575221239),
 u'agency': (u'adn', 644.109375),
 u'ago': (u'year', 139.22964258204533),
 u'agree': (u'principle', 334.3963636363637),
 u'agreement': (u'tentative', 577.8304511278195),
 u'aid': (u'infected', 1117.9554239401496),
 u'air': (u'langley', 955.8638059701492),
 u'airline': (u'traveler', 698.6495454545455),
 u'allow': (u'constitution', 76.92645329195966),
 u'along': (u'creek', 323.75545023696685),
 u'already': (u'recession', 193.2883551307847),
 u'also': (u'prices', 55.62899022801303),
 u'america': (u'latin', 953.6739690721649),
 u'american': (u'telegraph', 378.75932172064876),
 u'among': (u'chip', 174.98053278688525),
 u'ana

## Información Léxica[2]
Bajar de Project Gutenberg el libro de Darwin ON THE ORIGIN OF SPECIES.

#### 1. Procesar el texto, tokenizando eliminando signos de puntuación.

In [125]:
tokenizer = nk.tokenize.RegexpTokenizer(r'\w+')
with open('darwin.txt', 'r') as darwintexto:
    data= tokenizer.tokenize(darwintexto.read())

In [128]:
data = [d.lower() for d in data]
data

['on',
 'the',
 'origin',
 'of',
 'species',
 'introduction',
 'when',
 'on',
 'board',
 'h',
 'm',
 's',
 'beagle',
 'as',
 'naturalist',
 'i',
 'was',
 'much',
 'struck',
 'with',
 'certain',
 'facts',
 'in',
 'the',
 'distribution',
 'of',
 'the',
 'inhabitants',
 'of',
 'south',
 'america',
 'and',
 'in',
 'the',
 'geological',
 'relations',
 'of',
 'the',
 'present',
 'to',
 'the',
 'past',
 'inhabitants',
 'of',
 'that',
 'continent',
 'these',
 'facts',
 'seemed',
 'to',
 'me',
 'to',
 'throw',
 'some',
 'light',
 'on',
 'the',
 'origin',
 'of',
 'species',
 'that',
 'mystery',
 'of',
 'mysteries',
 'as',
 'it',
 'has',
 'been',
 'called',
 'by',
 'one',
 'of',
 'our',
 'greatest',
 'philosophers',
 'on',
 'my',
 'return',
 'home',
 'it',
 'occurred',
 'to',
 'me',
 'in',
 '1837',
 'that',
 'something',
 'might',
 'perhaps',
 'be',
 'made',
 'out',
 'on',
 'this',
 'question',
 'by',
 'patiently',
 'accumulating',
 'and',
 'reflecting',
 'on',
 'all',
 'sorts',
 'of',
 'facts',


#### 2. Siguiendo el artículo de la sección, calcular la autocorrelación para estimar la distribución de la palabra a lo largo del texto.

In [ ]:
laspalabra = ['for', 'plants', 'instinct']
for lapalabra in laspalabra:

#### 3.  Calcular la entrop´ıa de una selecci´on de 100 palabras que abarquen el rango de frecuencia de aparici´on en el libro, es decir elegir palabras que son muy frecuentes y otras de baja frecuencia.

#### 4. Calcular la entrop´ıa de las palabras seleccionadas anteriormente, randomizando su posici´on en el texto. Comparar con los resultados del punto anterior.

## Word embeddings, distancia semántica y WordNet

#### 1. Utilizando el test WordSim3531, comparar el rendimiento entre LSA[3] y Word2Vec2

#### 2. Comparar los distintos word embeddings con las medidas definidas en WordNet.